<a href="https://colab.research.google.com/github/Krishan-Kumar-kk/NLP/blob/main/Spam_classifier_LSTm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
df = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#filling NULL values with empty string
df=df.fillna('')
test=test.fillna('')

In [ ]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']

In [ ]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [ ]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg = X.copy()
msg_test = test.copy()

In [ ]:
#Downloading stopwords 
#Stopwords are the words in any language which does not add much meaning to a sentence.
#They can safely be ignored without sacrificing the meaning of the sentence.
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#We will be using Stemming here
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [ ]:
#Applying stemming and some preprocessing
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',msg['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
#Applying stemming and some preprocessing for test data
corpus_test = []
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [ ]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [ ]:
onehot_rep

[[989, 3887, 4101, 1944, 777, 1305, 1566, 3386, 3309, 4127, 712, 2664],
 [2358, 1907, 4493, 1495, 4703, 1348, 4713, 169, 1789, 2358],
 [1141, 3439, 586, 4201, 2533, 4686],
 [3565, 4345, 4461, 650, 3591, 2193, 242, 3001],
 [2998, 4703, 4645, 4863, 1705, 2421, 4703, 1029, 1994, 3794, 4404, 4362],
 [1682,
  4927,
  3735,
  2936,
  2162,
  2620,
  1985,
  2244,
  1662,
  3399,
  4948,
  2169,
  3586,
  1745,
  4713,
  169,
  4521],
 [4400, 4400, 122, 165, 4275, 1503, 799, 244, 3337, 1120, 4112, 657],
 [1060,
  4958,
  1980,
  2568,
  423,
  3987,
  4018,
  3802,
  3826,
  4429,
  4848,
  4501,
  1789,
  2250],
 [3690, 255, 4043, 1743, 2620, 4209, 4229, 4901, 3656, 2869, 3826, 4429, 4848],
 [4219,
  935,
  1252,
  4542,
  1024,
  4055,
  2620,
  1007,
  3826,
  4429,
  4848,
  528,
  3192,
  4411,
  1950],
 [2160, 4929, 4359, 2314, 3033, 4693, 200, 4707, 2620, 2771, 1946, 267],
 [644, 2107, 852, 4910, 4060, 3232, 1349, 3934, 3128, 1799],
 [449, 1822, 3206, 3996, 2893, 4782, 1488, 3451, 1473

In [ ]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)
print(embedded_docs)

[[   0    0    0 ... 4127  712 2664]
 [   0    0    0 ...  169 1789 2358]
 [   0    0    0 ... 4201 2533 4686]
 ...
 [   0    0    0 ... 2584  723 2795]
 [   0    0    0 ... 1831 1022 4464]
 [   0    0    0 ... 2660 3502 3768]]


In [ ]:
#We have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [ ]:
#Converting into numpy array
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (5200, 25))

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
#training model
model.fit(X_train,y_train,epochs=20,batch_size=64)

Epoch 1/20
218/218 [==============================] - 8s 32ms/step - loss: 0.0051 - accuracy: 0.9994
Epoch 2/20
218/218 [==============================] - 7s 32ms/step - loss: 0.0019 - accuracy: 0.9992
Epoch 3/20
218/218 [==============================] - 7s 32ms/step - loss: 4.7358e-04 - accuracy: 0.9999
Epoch 4/20
218/218 [==============================] - 7s 32ms/step - loss: 4.3170e-04 - accuracy: 0.9999
Epoch 5/20
218/218 [==============================] - 7s 32ms/step - loss: 1.0215e-04 - accuracy: 1.0000
Epoch 6/20
218/218 [==============================] - 7s 32ms/step - loss: 2.8746e-05 - accuracy: 1.0000
Epoch 7/20
218/218 [==============================] - 7s 32ms/step - loss: 1.4993e-04 - accuracy: 0.9999
Epoch 8/20
218/218 [==============================] - 7s 32ms/step - loss: 1.6712e-05 - accuracy: 1.0000
Epoch 9/20
218/218 [==============================] - 7s 32ms/step - loss: 3.4761e-05 - accuracy: 1.0000
Epoch 10/20
218/218 [==============================] - 7s 31ms/

In [ ]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [ ]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
pred = model.predict_classes(test_final)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
final_sub = pd.DataFrame()
final_sub['id']=test['id']
final_sub['label'] = pred
final_sub.to_csv('final_sub.csv',index=False)

In [ ]:
final_sub.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9988344988344988